<a href="https://colab.research.google.com/github/Renan-Domingues/LearnTheBasics-Pytorch/blob/main/Tutorials_04_BuidTheNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network
Neural Network performs operations in data.
The torch.nn provides all the buiding blocks needed to build my own neural network.
Every module in Pytorch subclasses the nn.Module. A neural network is a module itself that consists in other modules (layers)

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# Get device for training

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu")

print(f"device: {device}")

device: cuda


### Define the Class

We define the Neural network class by subclassing nn.Module, and initialize the layers in __init__.
Every nn.Module  subclass implements the operations on input data in the forward method.

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass the input data.
This executes the model's forward (along with some background operations)


Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 correponding to the individual values of each output.

We get the prediction probabilities by passing it through an instance of the nn.softmax module.

In [ ]:
X = torch.rand(1, 28, 28, device=device)

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8], device='cuda:0')


### Model Layer
Let's break down the layers in the FashionMNIST model
it will take a sample minibatch of 3 images (size 28*28)

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

we initialize the nn.flatten layer to convert each 2D 28*28 image into a contiguous array of 784 pixel values
the minibatch dimension (at dim=0) is mantained.

nn.Flaten = https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html

In [ ]:
flatten = nn.Flatten()
flat_image  = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

the linear layer is a module that aplies a linear transformation on the input using its stored weights and biases.

Linear = https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())


torch.Size([3, 20])


### nn.ReLU

non-linear activations are what create complex mappings between the model's inputs and outputs. They are applied after linear transformations to introduce nonlinearity

in this model we use nn.ReLU, but there are other activations

nn.ReLU = https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.8305,  0.3949,  0.2447,  0.9125,  0.2560,  0.1933,  0.2400, -0.2551,
         -0.1057,  0.1227, -0.4566,  0.1739,  0.3776, -0.4121,  0.0368,  0.4809,
         -0.5083, -0.0797,  0.4212,  0.4060],
        [ 0.4712,  0.0064,  0.2517,  0.4224, -0.2120, -0.1800,  0.3156, -0.2502,
         -0.1160, -0.1943, -0.1467, -0.2206,  0.4093, -0.6856, -0.2938,  0.4390,
         -0.5324, -0.1353,  0.2223,  0.0800],
        [ 0.4958, -0.0165,  0.2080,  0.7084, -0.0933,  0.0215,  0.3987, -0.1231,
          0.1478,  0.1490, -0.0511,  0.2866,  0.2841, -0.5656, -0.1398,  0.3751,
         -0.3462, -0.0108,  0.4068,  0.0234]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.8305, 0.3949, 0.2447, 0.9125, 0.2560, 0.1933, 0.2400, 0.0000, 0.0000,
         0.1227, 0.0000, 0.1739, 0.3776, 0.0000, 0.0368, 0.4809, 0.0000, 0.0000,
         0.4212, 0.4060],
        [0.4712, 0.0064, 0.2517, 0.4224, 0.0000, 0.0000, 0.3156, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.4093, 0.0000, 0.00

### nn.Sequential
The data passed trough all the modules in the same order as defined.
Is an ordered container of modules

Sequential = https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

The last linear layer return logits - raw values in [-infty,infty]- witch are passed to the nn.Softmax module.
The logits are scaled to values [0,1]crepresenting the model's predicted probabilities for each class.
dim oarameter indicates the dimension aling which values must sum to 1.

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model Parameters
Many layers inside a neural network have associated weights and biases that are optimized during training.
nn.Module makes all parameters accessible using your model's parameters() or named_parameters() methods.

in this exemple, we iterate over each parameter, and print its size and a preview of his values

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0017,  0.0035, -0.0144,  ...,  0.0094,  0.0340, -0.0293],
        [ 0.0073,  0.0325, -0.0280,  ...,  0.0123,  0.0250, -0.0312]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0100, -0.0160], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0214,  0.0243,  0.0407,  ...,  0.0210,  0.0338, -0.0249],
        [-0.0321,  0.0107, -0.0345,  ..., -0.0398, -0.0239,  0.0223]],
       device='cuda:0', grad_fn=<SliceBa